In [ ]:
!pip install --upgrade transformers datasets accelerate
from google.colab import drive
drive.mount('/content/drive')
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import transformers
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Only use the input text and hate label
df = pd.read_csv('/content/drive/MyDrive/train_with_topics.csv')
df = df[['text', 'topic_label', 'hate_label']].dropna()
df['hate_label'] = df['hate_label'].astype(int)

# Split into train/test
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
full_dataset = Dataset.from_pandas(df)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(example):
    return tokenizer(example['text'], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
full_dataset = full_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.rename_column("hate_label", "labels")
val_dataset = val_dataset.rename_column("hate_label", "labels")
full_dataset = full_dataset.rename_column("hate_label", "labels")

train_dataset.set_format("torch")
val_dataset.set_format("torch")
full_dataset.set_format("torch")


In [ ]:
# Load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/hate_model_basic_full_dataset",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    logging_dir="/content/logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train and evaluate
trainer.train()
trainer.evaluate()


In [ ]:
# Running model in-domain on Twitter dataset
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load test CSV
df = pd.read_csv("/content/drive/MyDrive/train.csv")

# Batch size for prediction
batch_size = 32

# Get the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)  # Ensure model is on the correct device

# Store predictions
all_preds = []
all_probs = []

# Iterate over the data in batches
for i in range(0, len(df), batch_size):
    batch = df[i : i + batch_size]

    test_encodings = tokenizer(
        batch["text"].tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**test_encodings)
        probs = torch.softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1)

    all_preds.extend(preds.cpu().numpy())
    all_probs.extend(probs.cpu().numpy().tolist())

# Add predictions to DataFrame
df["predicted_label"] = all_preds
df["predicted_prob"] = all_probs

# Calculate accuracy
correct = (df["predicted_label"] == df["hate_label"]).sum()
accuracy = correct / len(df)

# Print results
print(df[["text", "hate_label", "predicted_label"]].head())
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
# Running model out of domain on Reddit dataset

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load test CSV
df = pd.read_csv("/content/drive/MyDrive/test_reddit.csv")

# Get the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)  # Ensure model is on the correct device

# Batch size for prediction
batch_size = 32

# Store predictions
all_preds = []
all_probs = []

# Iterate over the data in batches
for i in range(0, len(df), batch_size):
    batch = df[i : i + batch_size]

    test_encodings = tokenizer(
        batch["text"].tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**test_encodings)
        probs = torch.softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1)

    all_preds.extend(preds.cpu().numpy())
    all_probs.extend(probs.cpu().numpy().tolist())

# Add predictions to DataFrame
df["predicted_label"] = all_preds
df["predicted_prob"] = all_probs

# Calculate accuracy
correct = (df["predicted_label"] == df["hate_label"]).sum()
accuracy = correct / len(df)

# Print results
print(df[["text", "hate_label", "predicted_label"]].head())
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
# Running model out of domain on Youtube dataset

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load test CSV
df = pd.read_csv("/content/drive/MyDrive/test_youtube.csv")

# Batch size for prediction
batch_size = 32

# Store predictions
all_preds = []
all_probs = []

# Iterate over the data in batches
for i in range(0, len(df), batch_size):
    batch = df[i : i + batch_size]

    test_encodings = tokenizer(
        batch["text"].tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**test_encodings)
        probs = torch.softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1)

    all_preds.extend(preds.cpu().numpy())
    all_probs.extend(probs.cpu().numpy().tolist())

# Add predictions to DataFrame
df["predicted_label"] = all_preds
df["predicted_prob"] = all_probs

# Calculate accuracy
correct = (df["predicted_label"] == df["hate_label"]).sum()
accuracy = correct / len(df)

# Print results
print(df[["text", "hate_label", "predicted_label"]].head())
print(f"Accuracy: {accuracy:.4f}")
